# Masked Evidence Lower Bound

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader

import numpy as np
import pandas as pd

import io
import os
import tqdm

In [2]:
def load_data(filepath: str):
    buffer = None
    filepath = os.path.expanduser(filepath)
    with open(filepath, "r") as f:
        buffer = io.StringIO(f.read().replace("TRUE", "1").replace("FALSE", "0"))
    return pd.read_csv(buffer).to_numpy()

In [18]:
mar60_mask = load_data("../Data Simulation/missingness mechanism level data/MNAR60_mask.csv")
mar60_full = load_data("../Data Simulation/missingness mechanism level data/MNAR60_X_full.csv")

In [19]:
test_train_mask = np.array(np.random.binomial(1, 1/3, (3000,)), dtype=np.bool)

In [20]:
mar60_full_test = mar60_full[test_train_mask, :]
mar60_full_train = mar60_full[test_train_mask ^ True, :]
mar60_mask_test = mar60_mask[test_train_mask, :]
mar60_mask_train = mar60_mask[test_train_mask ^ True, :]

In [21]:
# Model Hyperparameters

cuda = False
DEVICE = torch.device("cuda" if cuda else "cpu")

batch_size = 100

x_dim = 20
hidden_dim = 400
latent_dim = 200

lr = 1e-3

epochs = 30

In [22]:
kwargs = {'num_workers': 1, 'pin_memory': True}
test_loader = DataLoader((mar60_full_test, mar60_mask_test), batch_size=batch_size, shuffle=False, **kwargs)
train_loader = DataLoader((mar60_full_train, mar60_mask_train), batch_size=batch_size, shuffle=True, **kwargs)

In [23]:
class Encoder(nn.Module):
    """Gaussian MLP Encoder."""
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()

        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_mean = nn.Linear(hidden_dim, latent_dim)
        self.FC_var = nn.Linear(hidden_dim, latent_dim)

        self.LeakyReLU = nn.LeakyReLU(0.2)

        self.training = True

    def forward(self, x):
        h_ = self.LeakyReLU(self.FC_input(x))
        h_ = self.LeakyReLU(self.FC_input2(h_))
        mean = self.FC_mean(h_)
        log_var = self.FC_var(h_)

        return mean, log_var

In [24]:
class Decoder(nn.Module):
    """Gaussian MLP Decoder."""
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()

        self.FC_hidden = nn.Linear(latent_dim, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, output_dim)

        self.LeakyReLU = nn.LeakyReLU(0.2)

    def forward(self, x):
        h = self.LeakyReLU(self.FC_hidden(x))
        h = self.LeakyReLU(self.FC_hidden2(h))
        x_hat = torch.sigmoid(self.FC_output(h))
        return x_hat

In [25]:
class Model(nn.Module):
    def __init__(self, encoder, decoder):
        super(Model, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def reparametrization(self, mean, var):
        # Sample from standard normal
        epsilon = torch.randn_like(var).to(DEVICE)
        # Reparametrization trick
        z = mean + var * epsilon
        return z
    
    def forward(self, x):
        mean, log_var = self.encoder(x)
        # Takes exponential function (log_var -> var)
        z = self.reparametrization(mean, torch.exp(.5 * log_var))
        x_hat = self.decoder(z)

        return x_hat, mean, log_var

In [26]:
encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim=hidden_dim, output_dim=x_dim)

model = Model(encoder=encoder, decoder=decoder).to(DEVICE)

### 3. Define Loss Function

In [27]:
from torch.optim import Adam

BCE_loss = nn.BCELoss()


def loss_function(x, x_hat, mean, log_var, mask):
    reproduction_loss = nn.functional.binary_cross_entropy(
    x_hat,
    x.float(),
    reduction='none'
)
    masked_repr_loss = (x_hat * mask).sum()
    KLD = -.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())

    return masked_repr_loss + KLD


optimizer = Adam(model.parameters(), lr=lr)

### 4. Train the Model

In [28]:
model.train()

for epoch in range(epochs):
    overall_loss = 0
    for batch_idx, (x, masks), in enumerate(train_loader):
        original_x = x.clone()  # Save original x for loss computation
        
        x = x.to(DEVICE).float()
        masks = masks.to(DEVICE).float()

        x = x * masks
        x = x.to(DEVICE).float()

        optimizer.zero_grad()

        x_hat, mean, log_var = model(x)
        loss = loss_function(original_x, x_hat, mean, log_var, masks)

        overall_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"\tEpoch {epoch + 1} complete.\tAverage Loss: {overall_loss / (batch_idx + 1)}")
print("Finish.")

/home/polar0n/Documents/Classes/Research Seminar/animated-palm-tree/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


	Epoch 1 complete.	Average Loss: 12965.9921875
	Epoch 2 complete.	Average Loss: 11435.9853515625
	Epoch 3 complete.	Average Loss: 10753.2060546875
	Epoch 4 complete.	Average Loss: 9763.6826171875
	Epoch 5 complete.	Average Loss: 8618.2060546875
	Epoch 6 complete.	Average Loss: 7344.2587890625
	Epoch 7 complete.	Average Loss: 5661.4873046875
	Epoch 8 complete.	Average Loss: 4291.87548828125
	Epoch 9 complete.	Average Loss: 3361.4453125
	Epoch 10 complete.	Average Loss: 2312.8134765625
	Epoch 11 complete.	Average Loss: 1397.0665283203125
	Epoch 12 complete.	Average Loss: 892.0467529296875
	Epoch 13 complete.	Average Loss: 575.6755981445312
	Epoch 14 complete.	Average Loss: 356.8083801269531
	Epoch 15 complete.	Average Loss: 227.0950164794922
	Epoch 16 complete.	Average Loss: 159.65879821777344
	Epoch 17 complete.	Average Loss: 105.37462615966797
	Epoch 18 complete.	Average Loss: 68.76162719726562
	Epoch 19 complete.	Average Loss: 54.19976043701172
	Epoch 20 complete.	Average Loss: 48.591

In [ ]:
model.eval()

total_test_loss = 0.0
total_samples = 0

with torch.no_grad():
    for x, masks in tqdm.tqdm(test_loader):
        x = x.to(DEVICE).float()
        masks = masks.to(DEVICE).float()

        x_hat, mean, log_var = model(x)
        
        # Calculate loss for the batch
        batch_loss = loss_function(x, x_hat, mean, log_var, masks)

        total_test_loss += batch_loss.item() * x.size(0)
        total_samples += x.size(0)

# Calculate final average error metric
average_loss = total_test_loss / total_samples
print(f"Test Loss: {average_loss:.4f}")

100%|██████████| 1/1 [00:00<00:00,  8.57it/s]

Test Loss: 8.8128
